In [6]:
import json
import os
from PIL import Image
import numpy as np
import random


icons_to_swap = [
    "setting", "video cam", "pause",
    "emoji", "notifications"
]
 ,

source_dir = r"F:\prism\source"
dest_dir = r"F:\prism\destination_image"
source_json_dir = r"F:\prism\source_json"
dest_json_dir = r"F:\prism\destination_json"


for directory in [source_dir, dest_dir, source_json_dir, dest_json_dir]:
    os.makedirs(directory, exist_ok=True)


def calculate_distance(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

def min_distance_from_icons(candidate_position, existing_icons, min_distance):
    for icon in existing_icons:
        if 'bndbox' not in icon:
            continue
        icon_center = ((icon['bndbox']['xmin'] + icon['bndbox']['xmax']) / 2,
                       (icon['bndbox']['ymin'] + icon['bndbox']['ymax']) / 2)
        if calculate_distance(candidate_position, icon_center) < min_distance:
            return False
    return True

def does_not_overlap(test_bndbox, existing_bndboxes):
    for existing_bndbox in existing_bndboxes:
        if not all(key in existing_bndbox for key in ['xmin', 'ymin', 'xmax', 'ymax']):
            continue
        if test_bndbox['xmax'] <= existing_bndbox['xmin'] or \
           test_bndbox['xmin'] >= existing_bndbox['xmax'] or \
           test_bndbox['ymax'] <= existing_bndbox['ymin'] or \
           test_bndbox['ymin'] >= existing_bndbox['ymax']:
            return True
    return False

def append_to_json(data, name, xmin, ymin, xmax, ymax):
    new_entry = {"name": name, "bndbox": {"xmin": xmin, "ymin": ymin, "xmax": xmax, "ymax": ymax}}
    data["outputs"]["object"].append(new_entry)
    return data

def find_suitable_position(icon_image, destination_metadata, dest_img_path):
    
    if 'size' not in destination_metadata or \
       'width' not in destination_metadata['size'] or \
       'height' not in destination_metadata['size'] or \
       'outputs' not in destination_metadata or \
       'object' not in destination_metadata['outputs']:
        return None

    width, height = destination_metadata['size']['width'], destination_metadata['size']['height']
    positions = [(x, y) for x in range(0, width - icon_image.width, icon_image.width // 2)
                 for y in range(0, height - icon_image.height, icon_image.height // 2)]
    random.shuffle(positions)

    for pos in positions:
        test_bndbox = {'xmin': pos[0], 'ymin': pos[1], 'xmax': pos[0] + icon_image.width, 'ymax': pos[1] + icon_image.height}
        if does_not_overlap(test_bndbox, [icon['bndbox'] for icon in destination_metadata['outputs']['object'] if 'bndbox' in icon]):
            return pos
    return None

def process_images():
    source_filenames = np.array(sorted(os.listdir(source_dir), key=lambda x: int(x.split('.')[0])))
    dest_filenames = np.array(sorted(os.listdir(dest_dir), key=lambda x: int(x.split('.')[0])))
    icon_counter = {icon_name: 0 for icon_name in icons_to_swap}
    placed_icons = {dest_filename: None for dest_filename in dest_filenames}  

    
    for dest_filename in dest_filenames:
        dest_img_path = os.path.join(dest_dir, dest_filename)
        dest_json_path = os.path.join(dest_json_dir, dest_filename.replace('.jpg', '.json'))

        
        for icon_name in sorted(icons_to_swap, key=lambda x: icon_counter[x]):
            src_filename = find_source_with_icon(source_filenames, icon_name)
            if not src_filename:  
                continue

            src_img_path = os.path.join(source_dir, src_filename)
            src_json_path = os.path.join(source_json_dir, src_filename.replace('.jpg', '.json'))

            try:
                src_metadata = json.load(open(src_json_path, 'r'))
                src_image = Image.open(src_img_path)
                icon_data = next((item for item in src_metadata['outputs']['object'] if item['name'] == icon_name), None)
                icon_bndbox = icon_data['bndbox']
                icon_image = src_image.crop((icon_bndbox['xmin'], icon_bndbox['ymin'], icon_bndbox['xmax'], icon_bndbox['ymax'])).convert('RGBA')

                dest_metadata = json.load(open(dest_json_path, 'r'))
                best_position = find_suitable_position(icon_image, dest_metadata, dest_img_path)
                if best_position:
                    dest_image = Image.open(dest_img_path)
                    dest_image.paste(icon_image, best_position, icon_image)
                    updated_metadata = append_to_json(dest_metadata, icon_name, *best_position, best_position[0] + icon_image.width, best_position[1] + icon_image.height)
                    with open(dest_json_path, 'w') as djf:
                        json.dump(updated_metadata, djf, indent=4)
                    dest_image.save(dest_img_path)
                    icon_counter[icon_name] += 1  
                    placed_icons[dest_filename] = icon_name  
                    break  
            except Exception as e:
                print(f"An error occurred with {dest_filename} or {src_filename}: {e}")
            finally:
                src_image.close()

def find_source_with_icon(source_filenames, icon_name):
    
    for src_filename in source_filenames:
        src_json_path = os.path.join(source_json_dir, src_filename.replace('.jpg', '.json'))
        with open(src_json_path, 'r') as sjf:
            src_metadata = json.load(sjf)
        for icon in src_metadata['outputs']['object']:
            if icon['name'] == icon_name:
                return src_filename
    return None



def main():
    
    validate_directories([source_dir, dest_dir, source_json_dir, dest_json_dir])
    
    
    process_images()


def validate_directories(directories):
    for directory in directories:
        if not os.path.isdir(directory):
            print(f"Directory not found: {directory}")
            exit(1)

if __name__ == "__main__":
    main()

